# RAG

In [40]:
import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import unicodedata

import torch
import pandas as pd
from tqdm import tqdm

import re

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

from langchain_community.document_loaders import JSONLoader
from langchain_huggingface import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores.utils import DistanceStrategy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


## Config

In [55]:
CHUNK_SIZE = 256
CHUNK_OVERLAP = 128

EMBEDDING_MODEL = "intfloat/e5-large"

K = 3

LLM = "google/gemma-2-9b-it"

PROMPT_TEMPLATE = """
You are an AI visual assistant surveillance operator that can analyze real-time traffic analysis and accident detection.

Respond to user's questions as accurately as possible.
Be careful not to answer with false information.

Using the provided caption information, describe the scene in a detailed manner.
{context}

Question: {question}

Answer:
"""

QUANTIZATION = "bf16" # "qlora", "bf16", "fp16"

MAX_NEW_TOKENS = 512

## Vector DB

In [64]:
def process_json(file_path, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP):
    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".frame.[].caption",
        text_content=False,
    )
    docs = loader.load()
    
    video_doc = ""
    for doc in docs:
        video_doc += doc.page_content
        video_doc += '\n'
    
    meta_data = {
        'source' : file_path
    }
    
    chunks = [Document(page_content=video_doc, metadata=meta_data)]
    
    return chunks

def create_vector_db(chunks, model_path=EMBEDDING_MODEL):
    """FAISS DB"""
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def load_vector_db(chunks, model_path=EMBEDDING_MODEL):
    """FAISS DB"""
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.load_local(
        './db/th_v1',
        embeddings,
        allow_dangerous_deserialization=True
    )
    return db
    
def process_jsons_from_dataframe(unique_paths, base_directory):
    
    total_chunks = []
    
    for path in tqdm(unique_paths, desc="Processing JSONs"):
        
        normalized_path = unicodedata.normalize('NFC', path)
        full_path = os.path.normpath(
            os.path.join(
                base_directory, normalized_path.lstrip('./')
            )
        ) if not os.path.isabs(normalized_path) else normalized_path
        json_title = os.path.splitext(os.path.basename(full_path))[0]
        
        print(f"Processing {json_title}...")
        
        chunks = process_json(full_path)

        total_chunks.extend(chunks)
    
    db = create_vector_db(total_chunks)
    # db = load_vector_db(total_chunks)
    
    # Retriever

    retriever = db.as_retriever(
        search_type="similarity",
        search_kwargs={'k': K}
    )
    
    json_databases = {
            'db': db,
            'retriever': retriever
    }
    return json_databases

## DB 생성

In [65]:
# base_directory = 'data/msrvtt/' # Your Base Directory
# os.listdir(base_directory)

In [66]:
# file = open("data/MSR-VTT-1kA/val_list_jsfusion.txt", 'r')
# txt = file.read().split('\n')
# video_path = [idx + '.json' for idx in txt]
# len(video_path)

In [67]:
# video_path[:5]

In [68]:
%%time

base_directory = '/home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db' # Your Base Directory
db_list = [filename for filename in os.listdir(base_directory) if 'meta' in filename]
json_databases = process_jsons_from_dataframe(db_list, base_directory)

Processing JSONs: 100%|██████████| 4/4 [00:00<00:00, 138.90it/s]


Processing cam_07.mp4-meta_db...
Processing cam_06.mp4-meta_db...
Processing cam_04.mp4-meta_db...
Processing demo_1.mp4-meta_db...
CPU times: user 23 s, sys: 949 ms, total: 24 s
Wall time: 4.51 s


## DB Save and Load

In [71]:
# json_databases['db'].save_local('/home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/total_video')

model_path = EMBEDDING_MODEL
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

embeddings_model = HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

db = FAISS.load_local('/home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/total_video', embeddings_model, allow_dangerous_deserialization=True)
db

### example

In [77]:
prompt_sample = 'bike accident'

docs = json_databases['retriever'].invoke(prompt_sample)

for doc in docs:
    print(f"video_id : {doc.metadata['source']}")
    # print(doc.page_content)
    print("=========================================================")

video_id : /home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/cam_04.mp4-meta_db.json
video_id : /home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/cam_07.mp4-meta_db.json
video_id : /home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/cam_06.mp4-meta_db.json


In [38]:
prompt_sample = 'a little girl does gymnastics'

docs = json_databases['retriever'].invoke(prompt_sample)

for doc in docs:
    print(f"video_id : {doc.metadata['source']}")
    # print(doc.page_content)
    print("=========================================================")

video_id : /home/jiyul/SPS_JY/TeletoVision_demo/public/frontend/static/db/cam_05.mp4-meta_db.json


## Evaluate

In [11]:
import pickle

with open('data/MSR-VTT-1kA/raw-captions.pkl', 'rb') as f:
    rc = pickle.load(f)

with open('data/MSR-VTT-1kA/jsfusion_val_caption_idx.pkl', 'rb') as f:
    jvci = pickle.load(f)

len(rc.keys()), len(jvci)

labels = {}

for video_id, cap_id in jvci.items():
    cap = ' '.join(rc[video_id][cap_id])
    labels[video_id] = cap

len(labels)

1000

In [12]:
import numpy as np

def compute_recall_at_k(predictions, targets, k):
    """
    Recall@k 계산 함수
    
    Parameters:
    predictions (list of list): 각 query에 대한 예측된 결과 리스트
    targets (list): 각 query에 대한 실제 정답
    k (int): k 값
    
    Returns:
    float: Recall@k 값
    """
    assert len(predictions) == len(targets), "Predictions와 Targets의 길이는 동일해야 합니다."
    
    recall_count = 0
    for i in range(len(targets)):
        if targets[i] in predictions[i][:k]:
            recall_count += 1
            
    return recall_count / len(targets)

def compute_mean_rank(predictions, targets):
    """
    Mean Rank 계산 함수
    
    Parameters:
    predictions (list of list): 각 query에 대한 예측된 결과 리스트
    targets (list): 각 query에 대한 실제 정답
    
    Returns:
    float: Mean Rank 값
    """
    ranks = []
    for i in range(len(targets)):
        if targets[i] in predictions[i]:
            ranks.append(predictions[i].index(targets[i]) + 1)
        else:
            ranks.append(len(predictions[i]) + 1)  # 만약 정답이 없다면 max rank로 설정
            
    return np.mean(ranks)

def compute_median_rank(predictions, targets):
    """
    Median Rank 계산 함수
    
    Parameters:
    predictions (list of list): 각 query에 대한 예측된 결과 리스트
    targets (list): 각 query에 대한 실제 정답
    
    Returns:
    float: Median Rank 값
    """
    ranks = []
    for i in range(len(targets)):
        if targets[i] in predictions[i]:
            ranks.append(predictions[i].index(targets[i]) + 1)
        else:
            ranks.append(len(predictions[i]) + 1)  # 정답이 없으면 최대 rank로 설정
    
    return np.median(ranks)

# Recall@1, Recall@5, Recall@10 계산 함수
def compute_all_metrics(predictions, targets):
    """
    Recall@1, Recall@5, Recall@10, Median Rank, Mean Rank를 계산하는 함수
    
    Parameters:
    predictions (list of list): 각 query에 대한 예측된 결과 리스트
    targets (list): 각 query에 대한 실제 정답
    
    Returns:
    dict: 각 메트릭의 결과를 담은 사전
    """
    metrics = {
        'Recall@1': compute_recall_at_k(predictions, targets, 1),
        'Recall@5': compute_recall_at_k(predictions, targets, 5),
        'Recall@10': compute_recall_at_k(predictions, targets, 10),
        'Median Rank': compute_median_rank(predictions, targets),
        'Mean Rank': compute_mean_rank(predictions, targets)
    }
    return metrics

In [13]:
predictions = []
targets = []

for k, v in tqdm(labels.items()):
    
    docs = json_databases['retriever'].invoke(v)
    
    preds = []
    for doc in docs:
        normalized_path = unicodedata.normalize('NFC', doc.metadata['source'])
        full_path = os.path.normpath(
            os.path.join(
                base_directory, normalized_path.lstrip('./')
            )
        ) if not os.path.isabs(normalized_path) else normalized_path
        json_title = os.path.splitext(os.path.basename(full_path))[0]
        preds.append(json_title)
        
    predictions.append(preds)
    targets.append(k)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [05:24<00:00,  3.08it/s]


In [14]:
metrics = compute_all_metrics(predictions, targets)
print(metrics)

{'Recall@1': 0.318, 'Recall@5': 0.566, 'Recall@10': 0.663, 'Median Rank': 4.0, 'Mean Rank': 5.512}
